# Lab Exercise Solution
## De-Duping Data

-sandbox
##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Instructions

In this exercise, we're doing ETL on a file we've received from some customer. That file contains data about people, including:

* first, middle and last names
* gender
* birth date
* Social Security number
* salary

But, as is unfortunately common in data we get from this customer, the file contains some duplicate records. Worse:

* In some of the records, the names are mixed case (e.g., "Carol"), while in others, they are uppercase (e.g., "CAROL"). 
* The Social Security numbers aren't consistent, either. Some of them are hyphenated (e.g., "992-83-4829"), while others are missing hyphens ("992834829").

The name fields are guaranteed to match, if you disregard character case, and the birth dates will also match. (The salaries will match, as well,
and the Social Security Numbers *would* match, if they were somehow put in the same format).

Your job is to remove the duplicate records. The specific requirements of your job are:

* Remove duplicates. It doesn't matter which record you keep; it only matters that you keep one of them.
* Preserve the data format of the columns. For example, if you write the first name column in all lower-case, you haven't met this requirement.
* Write the result as a Parquet file, as designated by *destFile*.
* The final Parquet "file" must contain 8 part files (8 files ending in ".parquet").

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** The initial dataset contains 103,000 records.<br/>
The de-duplicated result haves 100,000 records.

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Getting Started

Run the following cell to configure our "classroom."

In [0]:
%run "../Includes/Classroom-Setup"

Initialized classroom variables & functions...

In [0]:
# Mount "/mnt/training" again using "%run "./Includes/Dataset-Mounts-New"" if it is failed in "./Includes/Classroom-Setup"
try:
    files = dbutils.fs.ls("/mnt/training")
except:
    dbutils.fs.unmount('/mnt/training/')


/mnt/training/ has been unmounted.


Mounted datasets to /mnt/training from wasbs://training@dbtrainwesteurope.blob.core.windows.net/

In [0]:
%run "../Includes/Dataset-Mounts-New"

Created user-specific database

Using the database vishal_abnave_borregaard_com_db .

All done!

In [0]:
%run "../Includes/Initialize-Labs"

Datasets are mounted


Initializing lab environment: 
 Declared clearYourResults( passedOnly:Boolean=true ) 
 Declared validateYourSchema( what:String, df:DataFrame, expColumnName:String, expColumnType:String ) 
 Declared validateYourAnswer( what:String, expectedHash:Int, answer:Any ) 
 Declared summarizeYourResults() 
 Declared logYourTest( path:String, name:String, value:Double ) 
 Declared loadYourTestResults( path:String ) returns DataFrame 
 Declared loadYourTestMap( path:String ) returns Map[String,Double]

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Hints

* Use the <a href="http://spark.apache.org/docs/latest/api/python/index.html" target="_blank">API docs</a>. Specifically, you might find 
  <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame" target="_blank">DataFrame</a> and
  <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions" target="_blank">functions</a> to be helpful.
* It's helpful to look at the file first, so you can check the format. `dbutils.fs.head()` (or just `%fs head`) is a big help here.

In [0]:
# ANSWER

sourceFile = "dbfs:/mnt/training/dataframes/people-with-dups.txt"

dbutils.fs.mkdirs(userhome)
destFile = userhome + "/people.parquet"

# In case it already exists
dbutils.fs.rm(destFile, True)

# First, let's see what the file looks like.
print(dbutils.fs.head(sourceFile))

[Truncated to first 65536 bytes]
firstName:middleName:lastName:gender:birthDate:salary:ssn
Emanuel:Wallace:Panton:M:1988-03-04:101255:935-90-7627
Eloisa:Rubye:Cayouette:F:2000-06-20:204031:935-89-9009
Cathi:Svetlana:Prins:F:2012-12-22:35895:959-30-7957
Mitchel:Andres:Mozdzierz:M:1966-05-06:55108:989-27-8093
Angla:Melba:Hartzheim:F:1938-07-26:13199:935-27-4276
Rachel:Marlin:Borremans:F:1923-02-23:67070:996-41-8616
Catarina:Phylicia:Dominic:F:1969-09-29:201021:999-84-8888
Antione:Randy:Hamacher:M:2004-03-05:271486:917-96-3554
Madaline:Shawanda:Piszczek:F:1996-03-17:183944:963-87-9974
Luciano:Norbert:Sarcone:M:1962-12-14:73069:909-96-1669
Newton:Jose:Piacente:M:1969-11-05:264422:967-61-8575
Gretta:Jennefer:Dipinto:F:1921-10-28:118497:907-49-2510
Jenni:Janella:Mcquiston:F:1969-07-11:137418:932-55-7164
Malena:Apryl:Kings:F:1980-01-08:98204:934-48-2334
Willy:Russell:Merker:M:1967-10-10:41026:992-83-4829
Jami:Allen:Mulkerin:F:1958-08-08:236024:934-86-4126
Fernando:Lowell:Zebell:M:1955-07-03:2

In [0]:
# ANSWER

# dropDuplicates() will likely introduce a shuffle, so it helps to reduce the number of post-shuffle partitions.
spark.conf.set("spark.sql.shuffle.partitions", 8)

In [0]:
# ANSWER

# Okay, now we can read this thing.

df = (spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .option("sep", ":")
    .csv(sourceFile)
)

In [0]:
# ANSWER
from pyspark.sql.functions import *

dedupedDF = (df
  .select(col("*"),
      lower(col("firstName")).alias("lcFirstName"),
      lower(col("lastName")).alias("lcLastName"),
      lower(col("middleName")).alias("lcMiddleName"),
      translate(col("ssn"), "-", "").alias("ssnNums")
      # regexp_replace(col("ssn"), "-", "").alias("ssnNums")
      # regexp_replace(col("ssn"), """^(\d{3})(\d{2})(\d{4})$""", "$1-$2-$3").alias("ssnNums")
   )
  .dropDuplicates(["lcFirstName", "lcMiddleName", "lcLastName", "ssnNums", "gender", "birthDate", "salary"])
  .drop("lcFirstName", "lcMiddleName", "lcLastName", "ssnNums")
)

In [0]:
# ANSWER

# Now we can save the results. We'll also re-read them and count them, just as a final check.
# Just for fun, we'll use the Snappy compression codec. It's not as compact as Gzip, but it's much faster.
(dedupedDF.write
   .mode("overwrite")
   .option("compression", "snappy")
   .parquet(destFile)
)
dedupedDF = spark.read.parquet(destFile)
print("Total Records: {0:,}".format( dedupedDF.count() ))

Total Records: 100,000


In [0]:
# ANSWER

display( dbutils.fs.ls(destFile) )

path,name,size,modificationTime
dbfs:/user/vishal.abnave@borregaard.com/people.parquet/_SUCCESS,_SUCCESS,0,1684687611000
dbfs:/user/vishal.abnave@borregaard.com/people.parquet/_committed_3768642801439434675,_committed_3768642801439434675,624,1684687611000
dbfs:/user/vishal.abnave@borregaard.com/people.parquet/_started_3768642801439434675,_started_3768642801439434675,0,1684687610000
dbfs:/user/vishal.abnave@borregaard.com/people.parquet/part-00000-tid-3768642801439434675-1d9b2ae9-f378-4d2e-8f98-d61ec80df5e1-252-1-c000.snappy.parquet,part-00000-tid-3768642801439434675-1d9b2ae9-f378-4d2e-8f98-d61ec80df5e1-252-1-c000.snappy.parquet,408085,1684687611000
dbfs:/user/vishal.abnave@borregaard.com/people.parquet/part-00001-tid-3768642801439434675-1d9b2ae9-f378-4d2e-8f98-d61ec80df5e1-253-1-c000.snappy.parquet,part-00001-tid-3768642801439434675-1d9b2ae9-f378-4d2e-8f98-d61ec80df5e1-253-1-c000.snappy.parquet,752260,1684687611000
dbfs:/user/vishal.abnave@borregaard.com/people.parquet/part-00002-tid-3768642801439434675-1d9b2ae9-f378-4d2e-8f98-d61ec80df5e1-254-1-c000.snappy.parquet,part-00002-tid-3768642801439434675-1d9b2ae9-f378-4d2e-8f98-d61ec80df5e1-254-1-c000.snappy.parquet,408748,1684687611000
dbfs:/user/vishal.abnave@borregaard.com/people.parquet/part-00003-tid-3768642801439434675-1d9b2ae9-f378-4d2e-8f98-d61ec80df5e1-255-1-c000.snappy.parquet,part-00003-tid-3768642801439434675-1d9b2ae9-f378-4d2e-8f98-d61ec80df5e1-255-1-c000.snappy.parquet,748279,1684687611000
dbfs:/user/vishal.abnave@borregaard.com/people.parquet/part-00004-tid-3768642801439434675-1d9b2ae9-f378-4d2e-8f98-d61ec80df5e1-256-1-c000.snappy.parquet,part-00004-tid-3768642801439434675-1d9b2ae9-f378-4d2e-8f98-d61ec80df5e1-256-1-c000.snappy.parquet,405791,1684687611000
dbfs:/user/vishal.abnave@borregaard.com/people.parquet/part-00005-tid-3768642801439434675-1d9b2ae9-f378-4d2e-8f98-d61ec80df5e1-257-1-c000.snappy.parquet,part-00005-tid-3768642801439434675-1d9b2ae9-f378-4d2e-8f98-d61ec80df5e1-257-1-c000.snappy.parquet,398710,1684687611000


##![Spark Logo Tiny](https://s3-us-west-2.amazonaws.com/curriculum-release/images/105/logo_spark_tiny.png) Validate Your Answer

At the bare minimum, we can verify that you wrote the parquet file out to **destFile** and that you have the right number of records.

Running the following cell to confirm your result:

In [0]:
files = dbutils.fs.ls(destFile)
display(files)

path,name,size,modificationTime
dbfs:/user/vishal.abnave@borregaard.com/people.parquet/_SUCCESS,_SUCCESS,0,1684687611000
dbfs:/user/vishal.abnave@borregaard.com/people.parquet/_committed_3768642801439434675,_committed_3768642801439434675,624,1684687611000
dbfs:/user/vishal.abnave@borregaard.com/people.parquet/_started_3768642801439434675,_started_3768642801439434675,0,1684687610000
dbfs:/user/vishal.abnave@borregaard.com/people.parquet/part-00000-tid-3768642801439434675-1d9b2ae9-f378-4d2e-8f98-d61ec80df5e1-252-1-c000.snappy.parquet,part-00000-tid-3768642801439434675-1d9b2ae9-f378-4d2e-8f98-d61ec80df5e1-252-1-c000.snappy.parquet,408085,1684687611000
dbfs:/user/vishal.abnave@borregaard.com/people.parquet/part-00001-tid-3768642801439434675-1d9b2ae9-f378-4d2e-8f98-d61ec80df5e1-253-1-c000.snappy.parquet,part-00001-tid-3768642801439434675-1d9b2ae9-f378-4d2e-8f98-d61ec80df5e1-253-1-c000.snappy.parquet,752260,1684687611000
dbfs:/user/vishal.abnave@borregaard.com/people.parquet/part-00002-tid-3768642801439434675-1d9b2ae9-f378-4d2e-8f98-d61ec80df5e1-254-1-c000.snappy.parquet,part-00002-tid-3768642801439434675-1d9b2ae9-f378-4d2e-8f98-d61ec80df5e1-254-1-c000.snappy.parquet,408748,1684687611000
dbfs:/user/vishal.abnave@borregaard.com/people.parquet/part-00003-tid-3768642801439434675-1d9b2ae9-f378-4d2e-8f98-d61ec80df5e1-255-1-c000.snappy.parquet,part-00003-tid-3768642801439434675-1d9b2ae9-f378-4d2e-8f98-d61ec80df5e1-255-1-c000.snappy.parquet,748279,1684687611000
dbfs:/user/vishal.abnave@borregaard.com/people.parquet/part-00004-tid-3768642801439434675-1d9b2ae9-f378-4d2e-8f98-d61ec80df5e1-256-1-c000.snappy.parquet,part-00004-tid-3768642801439434675-1d9b2ae9-f378-4d2e-8f98-d61ec80df5e1-256-1-c000.snappy.parquet,405791,1684687611000
dbfs:/user/vishal.abnave@borregaard.com/people.parquet/part-00005-tid-3768642801439434675-1d9b2ae9-f378-4d2e-8f98-d61ec80df5e1-257-1-c000.snappy.parquet,part-00005-tid-3768642801439434675-1d9b2ae9-f378-4d2e-8f98-d61ec80df5e1-257-1-c000.snappy.parquet,398710,1684687611000


In [0]:
finalDF = spark.read.parquet(destFile)
finalCount = finalDF.count()

clearYourResults()
validateYourAnswer("01 Expected 100000 Records", 972882115, finalCount)
summarizeYourResults()


01 Expected 100000 Records was correct, your answer: 100000


01 Expected 100000 Records:,passed,100000
